In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import keras.backend as K
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import *
from keras.losses import *
from keras.preprocessing.image import ImageDataGenerator
import skimage.io as io
import skimage.transform as tr
import skimage.color
import dask.array as da
import cv2
from glob import glob
from unet import UNet

In [10]:
def fix_mask(mask):
    mask[mask < 100] = 0.0
    mask[mask >= 100] = 255.0


def adjustData(img,mask):
    fix_mask(mask)
    img = img/255.
    mask = mask/255.
    return (img,mask)

In [11]:
# Loads the data 

def loadData():

    imagePath = 'C:/Users/wohlfart/Desktop/Datenset_iteration/total/images'
    maskPath = 'C:/Users/wohlfart/Desktop/Datenset_iteration/total/masks'

    image_generator = ImageDataGenerator().flow_from_directory(imagePath, target_size=(224,224), color_mode ='grayscale' ,batch_size=32, class_mode=None, shuffle=False, save_to_dir = 'C:/Users/wohlfart/Desktop/Output/images', seed=1)
    mask_generator =  ImageDataGenerator().flow_from_directory(maskPath, target_size=(224,224), color_mode ='grayscale', batch_size=32, class_mode=None, shuffle=False, save_to_dir = 'C:/Users/wohlfart/Desktop/Output/masks', seed=1)

    print (image_generator[0].shape)
    print (mask_generator[0].shape)

    train_generator = zip(image_generator, mask_generator)

    for (img,mask) in train_generator:
        img,mask = adjustData(img,mask)
        yield (img,mask)

trainGen =  loadData()

In [12]:
# Plots the images in jupyter notebook for comparison
i = 2
print(image_generator[0][0].shape)
plt.imshow(image_generator[0][i, ..., 0], cmap = 'gray')
plt.show()
plt.imshow(mask_generator[0][i, ..., 0], cmap='gray')
plt.show()

NameError: name 'image_generator' is not defined

In [13]:
# Defining the loss function 
def iou(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (intersection + 1.) / (K.sum(y_true_f) + K.sum(y_pred_f) + 1. - intersection)

def iou_loss(y_true, y_pred):
    return  -iou(y_true, y_pred)

In [17]:
# Defining the model 
from unet import UNet
model = None
model = UNet((224,224,1), 1, 16, 4, 2.0)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 224, 224, 1)  0                                            
__________________________________________________________________________________________________
conv2d_47 (Conv2D)              (None, 224, 224, 16) 160         input_3[0][0]                    
__________________________________________________________________________________________________
conv2d_48 (Conv2D)              (None, 224, 224, 16) 2320        conv2d_47[0][0]                  
__________________________________________________________________________________________________
max_pooling2d_9 (MaxPooling2D)  (None, 112, 112, 16) 0           conv2d_48[0][0]                  
__________________________________________________________________________________________________
conv2d_49 

In [11]:
#Altes Modell laden 
from keras.models import load_model 
model = load_model('ds_step08.h5', custom_objects={'iou_loss': iou_loss})
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 224, 224, 1)  0                                            
__________________________________________________________________________________________________
conv2d_70 (Conv2D)              (None, 224, 224, 16) 160         input_4[0][0]                    
__________________________________________________________________________________________________
conv2d_71 (Conv2D)              (None, 224, 224, 16) 2320        conv2d_70[0][0]                  
__________________________________________________________________________________________________
max_pooling2d_13 (MaxPooling2D) (None, 112, 112, 16) 0           conv2d_71[0][0]                  
__________________________________________________________________________________________________
conv2d_72 

In [18]:
model.compile(optimizer=Adam(lr=0.0001), loss=iou_loss)

In [19]:
hist = model.fit_generator(trainGen, steps_per_epoch= 21501//32, epochs=10, verbose=1, shuffle=False)

Epoch 1/8
671/671 [==============================] - 165s 247ms/step - loss: -0.0232
Epoch 2/8
671/671 [==============================] - 162s 242ms/step - loss: -5.6987e-05
Epoch 3/8
671/671 [==============================] - 162s 242ms/step - loss: -0.0044
Epoch 4/8
671/671 [==============================] - 162s 242ms/step - loss: -0.0182
Epoch 5/8
671/671 [==============================] - 162s 242ms/step - loss: -0.1278
Epoch 6/8
671/671 [==============================] - 162s 242ms/step - loss: -0.3534
Epoch 7/8
671/671 [==============================] - 162s 242ms/step - loss: -0.4358
Epoch 8/8
671/671 [==============================] - 163s 242ms/step - loss: -0.4777


In [100]:
model.save('ds_full.h5')